In [ ]:
import matplotlib.pyplot as plt
import matplotlib.widgets as wig

from typing import List, Type

%matplotlib widget
import arm_lib

In [ ]:
# close all previouse figure windows
plt.close("all")

### Plotting

In [ ]:
## setting up the plot
fig, ax = plt.subplots()
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim([-2, 2])
ax.set_ylim([0,2])
ax.set_aspect("equal")

# simple errored state
state = arm_lib.ArmState.OK
base = arm_lib.LimitedRevoluteJoint(0.0, 90.0, 80.0, [0])
first = arm_lib.LinearElement(1.0, 0.0, 0)
second = arm_lib.LimitedRevoluteJoint(-170.0, 90.0, -160.0, [0, 1])
third = arm_lib.LinearElement(0.5, 0.0, 1)
fourth = arm_lib.RevoluteJoint(0.0, [1, 0])
fifth = arm_lib.LinearElement(0.5, 0.0, 0)

joint_vector: List[Type[arm_lib.Joint]] = [
    base,
    first,
    second,
    third,
    fourth,
    fifth,
]

global_origin = (0.0, 0.0)
global_orientation = 0.0
global_frame = arm_lib.Frame2D(
    global_origin,
    global_orientation
)

arm = arm_lib.Arm(joint_vector, global_frame)
arm.initialize_plotters(ax)

# adjust the main plot to make room for the sliders
fig.subplots_adjust(bottom=0.25)
ax_base = fig.add_axes([0.2, 0.17, 0.65, 0.03])
base_angle = wig.Slider(
    ax=ax_base,
    label='base angle',
    valmin=0.0,
    valmax=180.0,
    valstep=0.1,
    valinit=base.rotation,
)

ax_first_rev = fig.add_axes([0.2, 0.12, 0.65, 0.03])
first_rev_angle = wig.Slider(
    ax=ax_first_rev,
    label='first angle',
    valmin=-180.0,
    valmax=180.0,
    valstep=0.1,
    valinit=second.rotation,
)

ax_second_rev = fig.add_axes([0.2, 0.07, 0.65, 0.03])
second_rev_angle = wig.Slider(
    ax=ax_second_rev,
    label='second angle',
    valmin=-720.0,
    valmax=720.0,
    valstep=0.5,
    valinit=fourth.rotation,
)

def update(angle: float):
    global state
    if state == arm_lib.ArmState.ERROR:
        return
    
    try:
        base.rotate(base_angle.val)
    except arm_lib.JointLimitError as error:
        ax.set_title(f"base: {error}", color='red')
        state = arm_lib.ArmState.ERROR
        arm.plot[0].error()
        fig.canvas.draw_idle()
        return

    try:
        second.rotate(first_rev_angle.val)
    except arm_lib.JointLimitError as error:
        ax.set_title(f"second: {error}", color='red')
        state = arm_lib.ArmState.ERROR
        arm.plot[2].error()
        fig.canvas.draw_idle()
        return
    
    fourth.rotate(second_rev_angle.val)
    
    arm.update_plotters()
    fig.canvas.draw_idle()

base_angle.on_changed(update)
first_rev_angle.on_changed(update)
second_rev_angle.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = fig.add_axes([0.8, 0.025, 0.1, 0.04])
button = wig.Button(resetax, 'Reset', hovercolor='0.975')

def reset(event):
    global state

    arm.plot[0].recover()
    arm.plot[2].recover()

    first_rev_angle.reset()
    base_angle.reset()
    second_rev_angle.reset()

    # only enable updates once all are cleared
    state = arm_lib.ArmState.OK
    # force update
    update(0.0)

    ax.set_title("")


button.on_clicked(reset)

plt.show()